In [ ]:

import os
import time
import numpy as np
from typing import List
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.embeddings import Embeddings
from langchain_pinecone import PineconeVectorStore
from huggingface_hub import InferenceClient

import chromadb


chroma_client = chromadb.Client()



# load the documents from the PDF directory
loader = PyPDFDirectoryLoader("./pdfs")
documents = loader.load()